In [ ]:
# CNN Project: Image Classification of Tomatoes, Cherries, and Strawberries

# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import random

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Seed for reproducibility
SEED = 309
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

# Data Preparation
data_path = './processed_data/'  # Update this path as necessary

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageFolder(root=data_path, transform=transform)

# Split dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Tracking metrics for final report
results = {"model_name": [], "epochs": [], "train_accuracy": [], "val_accuracy": []}


# Baseline CNN Model
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16 * 64 * 64, 3)  # Assumes final pooled image size is 64x64
        )

    def forward(self, x):
        return self.model(x)

baseline_model = BaselineCNN().to(device)


# Customizable CNN Model
class MultiLayerCNN(nn.Module):
    def __init__(self):
        super(MultiLayerCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 3)  # Adjust dimensions as needed
        )

    def forward(self, x):
        return self.model(x)

multi_layer_model = MultiLayerCNN().to(device)


# Training Function
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    train_losses, val_losses, train_acc, val_acc = [], [], [], []
    for epoch in range(num_epochs):
        model.train()
        correct_train, total_train, epoch_loss = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        
        train_losses.append(epoch_loss / len(train_loader))
        train_acc.append(100 * correct_train / total_train)

        # Validation step
        model.eval()
        correct_val, total_val, val_loss = 0, 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)
        
        val_losses.append(val_loss / len(test_loader))
        val_acc.append(100 * correct_val / total_val)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, '
              f'Val Loss: {val_losses[-1]:.4f}, Train Acc: {train_acc[-1]:.2f}%, Val Acc: {val_acc[-1]:.2f}%')
    
    return train_losses, val_losses, train_acc, val_acc


# Experiment: Baseline CNN Model with Different Optimizers
optimizers = {
    "SGD": optim.SGD(baseline_model.parameters(), lr=0.01),
    "Adam": optim.Adam(baseline_model.parameters(), lr=0.001)
}

for opt_name, optimizer in optimizers.items():
    print(f"Training Baseline CNN with {opt_name} Optimizer")
    model = BaselineCNN().to(device)
    train_losses, val_losses, train_acc, val_acc = train_and_evaluate(model, train_loader, test_loader, nn.CrossEntropyLoss(), optimizer)
    results['model_name'].append(f"Baseline CNN - {opt_name}")
    results['train_accuracy'].append(train_acc)
    results['val_accuracy'].append(val_acc)
    results['epochs'].append(list(range(1, len(train_acc) + 1)))


# Experiment: MultiLayer CNN with Different Activation Functions
activations = {"ReLU": nn.ReLU, "LeakyReLU": nn.LeakyReLU}

for act_name, act_fn in activations.items():
    model = MultiLayerCNN().to(device)
    model.model[1] = act_fn()  # First activation
    model.model[4] = act_fn()  # Second activation
    model.model[7] = act_fn()  # Third activation
    print(f"Training MultiLayer CNN with {act_name} Activation")
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_losses, val_losses, train_acc, val_acc = train_and_evaluate(model, train_loader, test_loader, nn.CrossEntropyLoss(), optimizer)
    results['model_name'].append(f"MultiLayer CNN - {act_name} Activation")
    results['train_accuracy'].append(train_acc)
    results['val_accuracy'].append(val_acc)
    results['epochs'].append(list(range(1, len(train_acc) + 1)))


# Experiment: Transfer Learning with ResNet50
transfer_model = resnet50(pretrained=True)
transfer_model.fc = nn.Linear(transfer_model.fc.in_features, 3)
transfer_model = transfer_model.to(device)

optimizer = optim.Adam(transfer_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("Training Transfer Learning Model - ResNet50")
train_losses, val_losses, train_acc, val_acc = train_and_evaluate(transfer_model, train_loader, test_loader, criterion, optimizer)
results['model_name'].append("Transfer Learning - ResNet50")
results['train_accuracy'].append(train_acc)
results['val_accuracy'].append(val_acc)
results['epochs'].append(list(range(1, len(train_acc) + 1)))


# Visualization of Results
plt.figure(figsize=(12, 8))
for i, name in enumerate(results['model_name']):
    plt.plot(results['epochs'][i], results['train_accuracy'][i], label=f'{name} - Train Acc')
    plt.plot(results['epochs'][i], results['val_accuracy'][i], linestyle='--', label=f'{name} - Val Acc')

plt.title("Training and Validation Accuracy across Different Configurations")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()


Using device: cuda
Training Baseline CNN with SGD Optimizer
Epoch [1/10], Train Loss: 1.1111, Val Loss: 1.1181, Train Acc: 32.02%, Val Acc: 33.49%
